In [109]:
import pandas as pd
from DLC_for_WBFM.utils.projects.utils_project import safe_cd, load_config
from DLC_for_WBFM.utils.pipeline.traces_pipeline import _unpack_configs_for_traces, get_traces_from_3d_tracks
import os
from pathlib import Path
import pickle
import matplotlib.pyplot as plt

In [130]:
project_dir = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-long"

with safe_cd(project_dir):
    name = 'neuron4207'
    fname = f"3-tracking/manual_tracking/{name}.csv"
    neuron1 = pd.read_csv(fname, header=[0,1], index_col=0)
    
    name = 'neuron4453'
    fname = f"3-tracking/manual_tracking/{name}.csv"
    neuron2 = pd.read_csv(fname, header=[0,1], index_col=0)
    
    name = 'neuron4528'
    fname = f"3-tracking/manual_tracking/{name}.csv"
    neuron3 = pd.read_csv(fname, header=[0,1], index_col=0)
    
    neuron_zxy = neuron1.join(neuron2)
    neuron_zxy = neuron_zxy.join(neuron3)
    neuron_zxy = neuron_zxy.replace(np.nan, 0.0)
    
    fname = r"3-tracking\manual_tracking\worm3_neurons.xlsx"
    manual_tracks = pd.read_excel(fname, sheet_name=name)
    
    project_cfg = load_config('project_config.yaml')
    
    segment_fname = Path(project_cfg['subfolder_configs']['segmentation'])
    segment_cfg = dict(load_config(segment_fname))
    
    track_fname = Path(project_cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))


## See get_traces_from_3d_tracks_using_config

In [131]:
with safe_cd(project_dir):
    _, green_fname, is_mirrored, mask_array, max_dist, num_frames, num_slices, params_start_volume, segmentation_metadata, z_to_xy_ratio = _unpack_configs_for_traces(
            project_cfg, segment_cfg, track_cfg)

In [132]:
# Use custom track (one neuron) instead of old ones
DEBUG = False
all_matches, all_neuron_names, green_dat, red_dat = get_traces_from_3d_tracks(DEBUG, neuron_zxy, green_fname,
                                                                                  is_mirrored, mask_array, max_dist,
                                                                                  num_frames, num_slices,
                                                                                  params_start_volume,
                                                                                  segmentation_metadata, z_to_xy_ratio)

  0%|                                                                                         | 0/1500 [00:00<?, ?it/s]c:\users\charles.fieseler\documents\current_work\dlc_for_wbfm\DLC_for_WBFM\utils\pipeline\traces_pipeline.py:119: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_matches[i_volume] = np.array([(m[0] + 1, m[1] + 1, c) for m, c in zip(matches, conf)])
  1%|▍                                                                                | 8/1500 [00:00<00:18, 79.55it/s]

Matching segmentation and DLC tracking...


  0%|                                                                                         | 0/1500 [00:00<?, ?it/s]

Extracting green traces...


100%|██████████████████████████████████████████████████████████████████████████████| 1500/1500 [32:44<00:00,  1.31s/it]


# Save

In [133]:
with safe_cd(project_dir):
    out_fname = "4-traces/retracked_neurons/green_traces.h5"
    green_dat.to_hdf(out_fname, "df_with_missing")
    
    out_fname = "4-traces/retracked_neurons/red_traces.h5"
    red_dat.to_hdf(out_fname, "df_with_missing")
    
    out_fname = "4-traces/retracked_neurons/matches.pickle"
    with open(out_fname, 'wb') as f:
        pickle.dump(all_matches, f)